<a href="https://colab.research.google.com/github/BaptisteZloch/Weather_forcasting/blob/main/Weather_forcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [115]:
#importing CSV file
#Reading file

df = pd.read_csv('/content/weather_2.csv',sep=";") #store our dataFrame into Df
df.head()

,MinTemp,MaxTemp,Rainfall,Sunshine,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,8.0,24.3,0.0,6.3,SW,NW,6.0,20,68,29,1019.7,1015.0,7,7,14.4,23.6,0,3.6,1
1,14.0,26.9,3.6,9.7,E,W,4.0,17,80,36,1012.4,1008.4,5,3,17.5,25.7,1,3.6,1
2,13.7,23.4,3.6,3.3,N,NNE,6.0,6,82,69,1009.5,1007.2,8,7,15.4,20.2,1,39.8,1
3,13.3,15.5,39.8,9.1,WNW,W,30.0,24,62,56,1005.5,1007.0,2,7,13.5,14.1,1,2.8,1
4,7.6,16.1,2.8,10.6,SSE,ESE,20.0,28,68,49,1018.3,1018.5,7,7,11.1,15.4,1,0.0,0


In [121]:
df = df[["MinTemp","MaxTemp","Rainfall","Sunshine","WindDir9am","WindDir3pm","WindSpeed9am","WindSpeed3pm","Humidity9am","Humidity3pm","Pressure9am","Pressure3pm","Cloud9am","Cloud3pm","Temp9am","Temp3pm","RainToday","RainTomorrow"]]

df["MinTemp"] = df["MinTemp"].astype(np.float64)
df["MaxTemp"] = df["MaxTemp"].astype(np.float64)
df["Rainfall"] = df["Rainfall"].astype(np.float64)
df["Sunshine"] = df["Sunshine"].astype(np.float64)
df["WindDir9am"] = df["WindDir9am"].astype(np.str)
df["WindDir3pm"] = df["WindDir3pm"].astype(np.str)
df["WindSpeed9am"] = df["WindSpeed9am"].astype(np.float64)
df["WindSpeed3pm"] = df["WindSpeed3pm"].astype(np.float64)
df["Humidity9am"] = df["Humidity9am"].astype(np.float64)
df["Humidity3pm"] = df["Humidity3pm"].astype(np.float64)
df["Pressure9am"] = df["Pressure9am"].astype(np.float64)
df["Pressure3pm"] = df["Pressure3pm"].astype(np.float64)
df["Cloud9am"] = df["Cloud9am"].astype(np.float64)
df["Cloud3pm"] = df["Cloud3pm"].astype(np.float64)
df["Temp9am"] = df["Temp9am"].astype(np.float64)
df["Temp3pm"] = df["Temp3pm"].astype(np.float64)
df["RainToday"] = df["RainToday"].astype(np.int64)
df["RainTomorrow"] = df["RainTomorrow"].astype(np.int64)

df.head()

,MinTemp,MaxTemp,Rainfall,Sunshine,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,8.0,24.3,0.0,6.3,SW,NW,6.0,20.0,68.0,29.0,1019.7,1015.0,7.0,7.0,14.4,23.6,0,1
1,14.0,26.9,3.6,9.7,E,W,4.0,17.0,80.0,36.0,1012.4,1008.4,5.0,3.0,17.5,25.7,1,1
2,13.7,23.4,3.6,3.3,N,NNE,6.0,6.0,82.0,69.0,1009.5,1007.2,8.0,7.0,15.4,20.2,1,1
3,13.3,15.5,39.8,9.1,WNW,W,30.0,24.0,62.0,56.0,1005.5,1007.0,2.0,7.0,13.5,14.1,1,1
4,7.6,16.1,2.8,10.6,SSE,ESE,20.0,28.0,68.0,49.0,1018.3,1018.5,7.0,7.0,11.1,15.4,1,0


Let's randomly shuffle the dataset by splitin it into a train set and a test set used to validate our model.

Train set = 85%

Test set = 15%


We do it by importing train_test_split from the Scikit-learn ML library.

In [122]:
#suffling data
from sklearn.model_selection import train_test_split

data_train ,data_test = train_test_split(df,test_size=0.15)

print("train set size : {}\ntest set size : {}".format(data_train.shape,data_test.shape))

train set size : (311, 18)
test set size : (55, 18)


In [123]:
data_train.head()

,MinTemp,MaxTemp,Rainfall,Sunshine,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
221,7.4,16.3,0.0,NaN,SSE,NE,6.0,19.0,75.0,65.0,1031.4,1027.9,7.0,7.0,12.1,14.5,0,1
218,4.4,16.7,0.0,9.0,SE,NNE,11.0,4.0,74.0,42.0,1025.2,1022.1,1.0,1.0,9.6,16.2,0,0
27,12.9,28.0,0.0,10.7,S,NNE,6.0,11.0,61.0,31.0,1019.2,1014.8,5.0,7.0,18.8,26.7,0,0
281,3.1,12.5,1.2,7.2,WNW,N,6.0,30.0,69.0,62.0,1019.8,1019.1,6.0,2.0,7.6,11.3,1,0
192,8.3,17.4,0.0,1.6,WSW,NE,6.0,11.0,80.0,52.0,1024.4,1021.5,7.0,7.0,13.5,17.2,0,0


Now we're going to extract our label from out data. We will select also the interesting columns by removing the labels.

In [124]:
tomorrow_label = data_train.pop('RainTomorrow')
today_label = data_train.pop('RainToday')
today_test_label = data_test.pop('RainToday')
tomorrow_test_label = data_test.pop('RainTomorrow')



#data_train = data_train[["MinTemp","MaxTemp","Rainfall","Sunshine","WindDir9am","WindDir3pm","WindSpeed9am","WindSpeed3pm","Humidity9am","Humidity3pm","Pressure9am","Pressure3pm","Cloud9am","Cloud3pm","Temp9am","Temp3pm"]]
#data_test = data_test[["MinTemp","MaxTemp","Rainfall","Sunshine","WindDir9am","WindDir3pm","WindSpeed9am","WindSpeed3pm","Humidity9am","Humidity3pm","Pressure9am","Pressure3pm","Cloud9am","Cloud3pm","Temp9am","Temp3pm"]]

data_train = data_train[["MinTemp","MaxTemp","Rainfall","Sunshine","WindSpeed9am","WindSpeed3pm","Humidity9am","Humidity3pm","Pressure9am","Pressure3pm","Cloud9am","Cloud3pm","Temp9am","Temp3pm"]]
data_test = data_test[["MinTemp","MaxTemp","Rainfall","Sunshine","WindSpeed9am","WindSpeed3pm","Humidity9am","Humidity3pm","Pressure9am","Pressure3pm","Cloud9am","Cloud3pm","Temp9am","Temp3pm"]]

We have to convert our data frame into a numpy array which is the only array taken by DNN.

In [125]:
data_train = np.array(data_train, dtype=np.float)
data_test = np.array(data_test, dtype=np.float)

today_label = np.array(today_label)
tomorrow_label = np.array(tomorrow_label)
today_test_label = np.array(today_test_label)
tomorrow_test_label = np.array(tomorrow_test_label)

from keras.utils import to_categorical
tomorrow_test_label=to_categorical(tomorrow_test_label)
today_test_label=to_categorical(today_test_label)

tomorrow_label=to_categorical(tomorrow_label)
today_label=to_categorical(today_label)

In [149]:
model_today = tf.keras.Sequential([
        tf.keras.layers.Dense(64,input_shape=(14,)),
        tf.keras.layers.Dense(2, activation="sigmoid")
])

model_today.summary()

model_today.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

hist = model_today.fit(x = data_train, y=today_label,epochs=100)

Epoch 1/100
10/10 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.8195
Epoch 2/100
10/10 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.8029
Epoch 3/100
10/10 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.8338
Epoch 4/100
10/10 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.8041
Epoch 5/100
10/10 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.8312
Epoch 6/100
10/10 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.8103
Epoch 7/100
10/10 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.8376
Epoch 8/100
10/10 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.8268
Epoch 9/100
10/10 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.8072
Epoch 10/100
10/10 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.8255
Epoch 11/100
10/10 [===================

In [138]:
print(model_today.predict(data_test[[6]]))

[[nan nan]]
